# 🔥 HeatMap – Sistema Inteligente de Combate a Queimadas
**Autores:** Matheus Farias - RM554254 / Miguel Parrado - RM554007

Este projeto simula uma central de emergência com múltiplas estruturas de dados para combate inteligente a queimadas, incluindo fila, heap de prioridade, pilha, lista ligada, árvore e algoritmo de Dijkstra.

In [1]:
from collections import deque
import heapq

# Fila FIFO
class CallQueue:
    def __init__(self):
        self.q = deque()
    def enqueue(self, call):
        self.q.append(call)
    def dequeue(self):
        return self.q.popleft() if self.q else None
    def __len__(self):
        return len(self.q)

# Heap de prioridade
class PriorityHeap:
    def __init__(self):
        self.h = []
    def push(self, priority, call):
        heapq.heappush(self.h, (priority, call))
    def pop(self):
        return heapq.heappop(self.h)[1] if self.h else None
    def __len__(self):
        return len(self.h)

# Pilha de ações
class ActionStack:
    def __init__(self):
        self.s = []
    def push(self, action):
        self.s.append(action)
    def list_actions(self):
        return list(reversed(self.s))

# Lista ligada
class AreaNode:
    def __init__(self, local, status):
        self.local = local
        self.status = status
        self.next = None

class AreaLinkedList:
    def __init__(self):
        self.head = None
    def update_status(self, local, status):
        n = self.head
        while n:
            if n.local == local:
                n.status = status
                return
            n = n.next
        novo = AreaNode(local, status)
        novo.next = self.head
        self.head = novo
    def status_dict(self):
        d, n = {}, self.head
        while n:
            d[n.local] = n.status
            n = n.next
        return d

# Árvore de regiões
class RegionTree:
    def __init__(self, name):
        self.name = name
        self.children = []
    def add_child(self, child):
        self.children.append(child)
    def display(self, level=0):
        print(" " * level * 2 + self.name)
        for child in self.children:
            child.display(level + 1)

In [2]:
# Algoritmo de Dijkstra
def dijkstra(graph, start, goal):
    heap = [(0, start, [])]
    visitados = set()
    while heap:
        dist, node, path = heapq.heappop(heap)
        if node in visitados:
            continue
        visitados.add(node)
        path = path + [node]
        if node == goal:
            return path, dist
        for viz, peso in graph.get(node, {}).items():
            if viz not in visitados:
                heapq.heappush(heap, (dist + peso, viz, path))
    return None, float("inf")

In [3]:
# Prioridade
VEG_WEIGHT = {"cerrado": 1.2, "mata_atlantica": 1.5, "pantanal": 2.0}
def calcular_prioridade(call):
    return call["severidade"] * VEG_WEIGHT.get(call["tipo_vegetacao"], 1)

In [4]:
# Função principal
def heatmap_resposta(chamadas, mapa, equipes):
    fila = CallQueue()
    heap = PriorityHeap()
    status = AreaLinkedList()
    resultados = []

    for c in chamadas:
        fila.enqueue(c)

    while len(fila):
        call = fila.dequeue()
        heap.push(calcular_prioridade(call), call)

    idx = 0
    while len(heap):
        call = heap.pop()
        equipe = equipes[idx % len(equipes)]
        idx += 1

        stack = ActionStack()
        stack.push("Criar aceiro")
        stack.push("Aplicar barreira de contenção")

        rota, tempo = dijkstra(mapa, "Base Central", call["local"])
        status.update_status(call["local"], "controle em andamento")

        resultado = {
            "ocorrencia_id": call["id"],
            "prioridade": round(calcular_prioridade(call), 2),
            "equipe": equipe,
            "acao": stack.list_actions(),
            "rota": rota,
            "tempo_estimado": tempo,
            "status_area": status.status_dict()[call["local"]]
        }

        print(f"🔥 Ocorrência {resultado['ocorrencia_id']} atendida:")
        print(f"  Prioridade: {resultado['prioridade']}")
        print(f"  Equipe designada: {resultado['equipe']}")
        print(f"  Rota: {' -> '.join(resultado['rota'])} (Tempo: {resultado['tempo_estimado']} min)")
        print(f"  Ações: {', '.join(resultado['acao'])}")
        print(f"  Status Atualizado: {resultado['status_area']}")
        print("-" * 60)

        resultados.append(resultado)

    return resultados

In [5]:
# Simulação
chamadas = [
    {"id": 1, "local": "Zona Norte", "severidade": 4, "tipo_vegetacao": "cerrado", "clima": "seco"},
    {"id": 2, "local": "Mata Alta", "severidade": 5, "tipo_vegetacao": "pantanal", "clima": "seco"},
    {"id": 3, "local": "Vila Verde", "severidade": 3, "tipo_vegetacao": "mata_atlantica", "clima": "úmido"}
]

mapa = {
    "Base Central": {"Zona Norte": 10, "Vila Verde": 5},
    "Zona Norte": {"Base Central": 10, "Mata Alta": 7},
    "Vila Verde": {"Base Central": 5, "Mata Alta": 3},
    "Mata Alta": {"Zona Norte": 7, "Vila Verde": 3}
}

equipes = ["Equipe Alpha", "Equipe Bravo", "Equipe Charlie"]
resultados = heatmap_resposta(chamadas, mapa, equipes)

🔥 Ocorrência 3 atendida:
  Prioridade: 4.5
  Equipe designada: Equipe Alpha
  Rota: Base Central -> Vila Verde (Tempo: 5 min)
  Ações: Aplicar barreira de contenção, Criar aceiro
  Status Atualizado: controle em andamento
------------------------------------------------------------
🔥 Ocorrência 1 atendida:
  Prioridade: 4.8
  Equipe designada: Equipe Bravo
  Rota: Base Central -> Zona Norte (Tempo: 10 min)
  Ações: Aplicar barreira de contenção, Criar aceiro
  Status Atualizado: controle em andamento
------------------------------------------------------------
🔥 Ocorrência 2 atendida:
  Prioridade: 10.0
  Equipe designada: Equipe Charlie
  Rota: Base Central -> Vila Verde -> Mata Alta (Tempo: 8 min)
  Ações: Aplicar barreira de contenção, Criar aceiro
  Status Atualizado: controle em andamento
------------------------------------------------------------
